# Unit Model Configuration

When undertaking a modeling exercise for a particular process, there can be any number of different approaches, assumptions, or relevant phenomena to include (or not include) in the model that is used. In WaterTAP, these discrete decisions are made at the point of model instantiation via passing of _configuration arguments_.

Passing different configuration arguments will typically result in the creation (or exclusion) of specific variables and constraints.

Some models have many configuration arguments and some have none. All WaterTAP unit process models require passing the required property model via the `property_package` configuration argument. This tutorial will go through configuration arguments for the WaterTAP pump and RO model.

<center><img src="graphics/config_arguments.png" width="700" /></center>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    value,
    assert_optimal_termination,
    units as pyunits,
)

# Imports from IDAES
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock

from watertap.unit_models import Pump, ReverseOsmosis0D
from watertap.core.solvers import get_solver

solver = get_solver()

# Pump Model & Configuration

Every WaterTAP model has default values for every configuration argument (except `property_package`). Using different model configurations will create additional variables and constraints that require additional parameter data from the user. 

### We will use the following parameters in our pump models

In [ ]:
# Pump parameters
eta = 0.8
bep_flow = flow = 1e-3
salinity = 35  # g/L
temperature = 298  # K
inlet_pressure = 40 * pyunits.bar
deltaP = 10 * pyunits.bar

flows = np.linspace(6e-4, bep_flow, 25)  # m3/s

### Create model, flowsheet, and add property model

Every WaterTAP model will begin with these three steps. 

The first flowsheet we are going to build in this tutorial is a pump without variable efficiency.

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

### Import pump and add to flowsheet

Any WaterTAP unit model can be imported with 

```python
from watertap.unit_models import ___________
```

Similarly, any model is added to the flowsheet with 

```python
m.fs.unit = UnitModel(
    property_package=m.fs.properties, 
    config_arg1="demo_config1",
    config_arg2="demo_config2"
)
```

The configuration arguments are contained in the `config` property on every unit model. You can view all possible configuration arguments using `.display()` on this property.

In [ ]:
from watertap.unit_models import Pump

# Add pump model to flowsheet
m.fs.pump = Pump(property_package=m.fs.properties)

# View configuration options
m.fs.pump.config.display()

### Fix inlet conditions

With the model added to the flowsheet, we can start fixing variables. The model needs to have zero degrees of freedom before solving.

Here, we start by fixing the state variables on the inlet state block. Most WaterTAP property models have the following state variables

- mass flow of all components
- temperature
- pressure

In [ ]:
# Fix inlet conditions
m.fs.pump.control_volume.properties_in[0].flow_vol_phase["Liq"].fix(flow)
m.fs.pump.control_volume.properties_in[0].conc_mass_phase_comp["Liq", "TDS"].fix(
    salinity
)
m.fs.pump.control_volume.properties_in[0].pressure.fix(inlet_pressure)
m.fs.pump.control_volume.properties_in[0].temperature.fix(temperature)

### Fix pump performance parameters

Next we will fix the other variables on the unit model. For the pump, that is the efficiency as `efficiency_pump` and the ∆P from inlet to outlet as `deltaP`.

In [ ]:
# Fix pump parameters
m.fs.pump.efficiency_pump.fix(eta)
m.fs.pump.deltaP.fix(deltaP)

### Solve the model

Lastly, we will check that we have zero degrees of freedom and solve the model.

The pumping power required is available as `work_mechanical` on the pump unit model.

In [ ]:
# Check degrees of freedom
assert degrees_of_freedom(m.fs.pump) == 0
# Solve model
results = solver.solve(m)
assert_optimal_termination(results)
m.fs.pump.work_mechanical.display()

## Build pump model with variable efficiency

The example below presents the `Pump` model build with `variable_efficiency`. Because we have added complexity to the model by introducing variable efficiency, this build creates three additional variables (`bep_flow`, `bep_eta`, and `flow_ratio`) and one additional constraint (`flow_ratio_constraint`).

We start this build the same as we start every WaterTAP flowsheet: model, flowsheet, property model.

In [ ]:
##################################################################
# Build Pump with variable_efficiency

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

### Configuring the variable efficiency pump

In this build, we are going to set the pump efficiency to be a function of the flow rate. We do this by passing `"flow"` to the configuration argument `variable_efficiency`.

In [ ]:
# Add pump model to flowsheet
# Set variable_efficiency to "flow"
m.fs.pump = Pump(property_package=m.fs.properties, variable_efficiency="flow")

### Fix inlet conditions and pump performance parameters and solve

Like before, we fix the inlet state variables and pump performance variables `deltaP`, `bep_eta`, and `bep_flow`. The variable `efficiency_pump` is now a function of these variables.

In [ ]:
# Fix inlet conditions
m.fs.pump.control_volume.properties_in[0].flow_vol_phase["Liq"].fix(flow)
m.fs.pump.control_volume.properties_in[0].conc_mass_phase_comp["Liq", "TDS"].fix(
    salinity
)
m.fs.pump.control_volume.properties_in[0].pressure.fix(inlet_pressure)
m.fs.pump.control_volume.properties_in[0].temperature.fix(temperature)

# Fix pump parameters
m.fs.pump.deltaP.fix(deltaP)
m.fs.pump.bep_eta.fix(eta)
m.fs.pump.bep_flow.fix(bep_flow)

### Solve to determine pump efficiency and power required

In [ ]:
# Solve model
assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

print(f"Pump efficiency: {value(m.fs.pump.efficiency_pump[0]):.4f}")
print(f"Pump power: {value(m.fs.pump.work_mechanical[0]):.4f}")

# RO Model & Configuration

The RO unit model has several different configuration arguments:

-`concentration_polarization_type`: none (ignore), fixed by user, or calculated by model

-`mass_transfer_coefficient`: none (ignore), fixed by user, or calculated by model

-`transport_model`: surface-diffusion (SD) or Speigler-Kedem-Katchalsky (SKK)

-`module_type`: flat sheet or spiral wound

-`pressure_change_type`: none (ignore) or fixed by user

In this exercise, we are going to present a simple and complex build for the RO model. Both will use spiral wound modules and the surface-diffusion model.

### *Simple*
```python
concentration_polarization_type="none"
mass_transfer_coefficient="none"
pressure_change_type="fixed_per_stage"
transport_model="SD"
module_type="spiral_wound"
```
### *Complex*
```python
concentration_polarization_type="calculated"
mass_transfer_coefficient="calculated"
pressure_change_type="calculated"
transport_model="SD"
module_type="spiral_wound"
```

### We will use the following parameters in our RO models

In [ ]:
inlet_pressure = 50 * pyunits.bar
temperature = 298  # K
A_comp = 4.2e-12
B_comp = 3e-8
membrane_area = 50  # m2
atmospheric = 101325  # Pa
deltaP = -3 * pyunits.bar
channel_height = 1 * pyunits.mm
spacer_porosity = 0.75

### RO Simple Model Build

We start the build:

1. create model
2. add flowsheet
3. add property model
4. add unit models

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="none",
    mass_transfer_coefficient="none",
    has_pressure_change=False,
    transport_model="SD",
    module_type="spiral_wound",
)

### Fix state variables

State variables for the RO model are on the `feed_side` sub-block of the main RO model.

In [ ]:
m.fs.RO.feed_side.properties_in[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.035)
m.fs.RO.feed_side.properties_in[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)

### Fix process variables

In addition to membrane properties and area, permeate side pressure is also a free variable in WaterTAP 

In [ ]:
m.fs.RO.permeate.pressure[0].fix(atmospheric)
m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)

print(f"Degrees of freedom: {degrees_of_freedom(m)}")

### Scale the model

Scaling ensures that all numerical quantities fall within similar, moderate magnitudes, improving model stability.

First we set default scaling factors for our state variables. Then we set a custom scaling factor for the membrane area. Lastly, we use `calculate_scaling_factors` which will set scaling factors for the remaining variables on the model.

In [ ]:
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

### Initialize & solve

Initialization helps the model get to an good initial point before sending it to the solver.

In [ ]:
m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

### Complex RO model build

We change the concentration polarization, mass transfer coefficient, and pressure change to be `"calculated"`. 

The model now has three additional degrees of freedom that must be specified: `deltaP`, `channel_height`, and `spacer_porosity`.

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="calculated",
    mass_transfer_coefficient="calculated",
    has_pressure_change=True,
    pressure_change_type="calculated",
    transport_model="SD",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)

m.fs.RO.permeate_side[0, 0].pressure.fix(atmospheric)
m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)
# Additional degrees of freedom from complex build
m.fs.RO.deltaP.fix(deltaP)
m.fs.RO.feed_side.channel_height.fix(channel_height)
m.fs.RO.feed_side.spacer_porosity.fix(spacer_porosity)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

# Try It Yourself

Create a complex RO model except use the Speigler-Kedem-Katchalsky (`"SKK"`) transport model and a `flat_sheet` module type.

Then, display the `config` dictionary and check the degrees of freedom.

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

# Create a "complex" RO model but with SKK transport model and a flat_sheet module


# Display the config dictionary

# Check the degrees of freedom

# Spiral Wound: Simple vs. Complex

This cell presents a comparison of volumetric recovery and permeate concentration using these two modeling approaches.

In [ ]:
pressures = np.linspace(35e5, 85e5, 10)

##################################################################
# SIMPLE BUILD
# RO without concentration polarization, mass transfer coefficient, or pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="none",
    mass_transfer_coefficient="none",
    has_pressure_change=False,
    pressure_change_type="fixed_per_stage",
    transport_model="SD",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate.pressure[0].fix(atmospheric)

m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)

print(f"Degrees of freedom: {degrees_of_freedom(m)}")

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

recovery1 = []
perm_conc1 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery1.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc1.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )


##################################################################
# COMPLEX BUILD - SD
# RO with concentration polarization, mass transfer coefficient, and pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="calculated",
    mass_transfer_coefficient="calculated",
    has_pressure_change=True,
    pressure_change_type="calculated",
    transport_model="SD",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate_side[0, 0].pressure.fix(atmospheric)

m.fs.RO.area.fix(membrane_area)

m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)
m.fs.RO.deltaP.fix(deltaP)
m.fs.RO.feed_side.channel_height.fix(channel_height)
m.fs.RO.feed_side.spacer_porosity.fix(spacer_porosity)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

m.fs.RO.initialize()

results = solver.solve(m)
assert_optimal_termination(results)

recovery2 = []
perm_conc2 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery2.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc2.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )

##################################################################
# Plot results

fig, ax = plt.subplots(2, 1, figsize=(6, 7), sharex=True)
ax[0].plot(
    pressures / 1e5,
    recovery1,
    marker="o",
    color="blue",
    label="Spiral Wound: Without CP, MTC, or ∆P",
)
ax[0].plot(
    pressures / 1e5,
    recovery2,
    marker="o",
    color="red",
    label="Spiral Wound: With CP, MTC, and ∆P",
)
ax[0].set_ylabel("Volumetric Recovery")
ax[0].set_title("RO Recovery vs Feed Pressure")
ax[0].grid(visible=True)
ax[0].legend()

ax[1].plot(
    pressures / 1e5,
    perm_conc1,
    marker="o",
    color="blue",
)
ax[1].plot(
    pressures / 1e5,
    perm_conc2,
    marker="o",
    color="red",
)
ax[1].set_xlabel("Feed Pressure (bar)")
ax[1].set_ylabel("TDS Concentration in Permeate (g/L)")
ax[1].set_title("Permeate Concentration vs Feed Pressure")
ax[1].grid(visible=True)

fig.tight_layout()